# 1. Order libraries 

In [ ]:
library(dplyr)
library(keras)
library(ggplot2)

In [ ]:
data = read.csv('/kaggle/input/imdb-50k-cleaned-movie-reviews/IMDB_cleaned.csv')

In [ ]:
data

In [ ]:
str(data)

In [ ]:
# create column from sediment column and must have a number row 
data$sentiment_numeric <- ifelse(data$sentiment == 'positive', 1,0)

# 2.plot data 

In [ ]:
barplot(table(data$sentiment_numeric))

# 3. Transforming data to tokenizers

In [ ]:
# check number words on columns and fitting 
text <- data$cleaned_review
tokeniszer <- text_tokenizer(num_words = 100 , oov_token = '<oov>') %>%
fit_text_tokenizer(text)


In [ ]:
# plot words but first transformer words to numbers understand how 
word_counts <- (tokeniszer$word_index)
barplot(as.numeric(word_counts), names.arg = names(word_counts))

In [ ]:
# This transformer converts words to numbers for the model. 
texts <- data$cleaned_review
sequences <- texts_to_sequences(tokeniszer , texts) %>%
    pad_sequences(maxlen = 10 , padding = 'post', truncating = 'post')

In [ ]:
str(sequences)

# 

# 4. build model 

In [ ]:
model <- keras_model_sequential() %>%
    layer_embedding(input_dim = length(tokeniszer$word_index) + 1 , output_dim = 16 , input_length = 10 )%>%
    layer_flatten() %>%
    layer_dense(units = 1 , activation  = 'sigmoid')

In [ ]:
model %>% compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = c('accuracy'))

In [ ]:
summary(model)

In [ ]:
# train the model 
model %>% fit( x = sequences , y = as.array(data$sentiment_numeric), epochs = 50 , verbose = 3)

In [ ]:
# predicting model
predictions <- predict(model , sequences)
summary(predictions)

In [ ]:
#classification scores model
predict_class <- ifelse(predictions > 0.6 , 1 , 0)

In [ ]:
barplot(table(predict_class))